Convolution Neural Network 

In [2]:
# cnn의 일반적인 구조 
# affine-ReLU 연결 -> Conv-ReLU-Pooling 연결로 변경
# 출력이 가까운 층에서는 affine-ReLU 구성이 그대로 사용됨
# 마지막 출력 층에서는 affine-Softmax 조합이 사용됨

# Affine 계층의 문제점 
# 입력 데이터의 형상이 무시됨 - 기존 affine 계층에서는 1차원의 데이터만 가능
# 합성곱 계층에서는 형상이 유지됨 - 합성곱 계층의 입출력 데이터를 특징 맵(Feature map)이라고 함

# 합성곱 연산 
# 입력 데이터에 필터를 적용
# 필터를 커널이라 하기도 함
# 필터의 윈도우를 일정한 간격으로 이동해가며 입력 데이터에 적용
# CNN의 가중치 매개변수와 편향
# 가중치 매개변수 : 필터의 매개변수 
# 편향은 항상(1x1)하나만 존재하며, 그 값을 필터를 적용한 모든 원소에 더함 

# 패딩 
# 합성곱 연산을 하기 전 입략 데이터 주변을 특정 값(ex.0)으로 채우는 작업

# 스트라이드 
# 필터를 적용하는 위치의 간격
# ex 스트라이드를 2로 하면 필터를 적용하는 윈도우가 2칸씩 이동

# 3차원 데이터 합성곱 연산 
# 2차원(가로x세로) + 채널
# 채널마다 입력데이터와 필터 간의 합성곱 연산을 수행하고, 그 결과를 더해서 하나의 출력을 얻음
# 입력 데이터와 필터의 채널 수가 같아야 함
# 모든 필터는 같은 크기여야 함
# 입력 크기 (C,H,W) * 필터 (C, FH, FW) -> 출력 데이터(1,OH, OW)
# 입력 크기 (C,H,W) * 필터 (C, FH, FW) (FN개) -> (FN ,OH, OW) + (FN , 1, 1)-> (FN,OH,OW) 출력데이터
# 입력 크기 (N,C,H,W)(N개) * 필터 (FN, C, FH, FW) (FN개) -> (N, FN ,OH, OW) + (FN , 1, 1)-> (N, FN,OH,OW) 출력데이터

# 풀링 계층
# 세로,가로 방향의 공간을 줄이는 연산
# 특정 윈도우에서 최대값이나 평균값을 선택하여 그 영역의 대표값으로 사용
# 윈도우와 스트라이드는 같은 값으로 설정
# 최대 풀링 : 특정 윈도우에서 최대값을 선택하는 방법
# 풀링의 특징:
#     학습해야할 매개변수가 없음
#     채널 수가 변하지 않음
#     입력의 변화에 영향을 적게 박음(강건성, Robustness)

합성곱/ 풀링계층 구현

In [3]:
import numpy as np
x = np.random.rand(10,1,28,28)
#높이 28, 너비 28, 채널 1, 데이터 수 10
x.shape

(10, 1, 28, 28)

In [4]:
#numpy에서는 for문을 가급적 사용하지 않는 것이 좋음
#im2col(image to column)
#입력 데이터에서 필터를 적용하는 영역을 한 줄로 바꿈
#im2col(image to culmn)사용 시 원소 수가 증가함 
#입력 데이터에서 필터를 적용하는 영역을 한 줄로 바꿈
#보통은 스트라이드를 크게 잡지 않기 때문에, 필터 적용 영역이 겹치게 됨

In [5]:
#input_data: (데이터 수, 채널 수, 높이, 너비)의 4차원 배열로 이루어진 입력 데이터
#filter_h : 필터의 높이 , filter_w: 필터의 너비 , stride : 스트라이드 , pad :패딩
def im2col(input_data, filter_h, filter_w, stride =1, pad =0):
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    img = np.pad(input_data, [(0,0),(0,0),(pad,pad),(pad,pad)], 'constant')
    col = np.zeros((N,C,filter_h,filter_w,out_h,out_w))
    
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
            
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col
# 한번 더 확인해보기

In [6]:
import sys, os
sys.path.append(os.pardir)
#from common.util import im2col

x1 = np.random.rand(1,3,7,7)#(데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1,5,5, stride = 1, pad = 0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)#(데이터 수, 채널 수, 높이, 너비)
col2 = im2col(x2,5,5, stride = 1, pad = 0)
print(col2.shape)

(9, 75)
(90, 75)


In [7]:
#합성곱 계층 구현 
class Convolution:
    def __init__(self, W, b, stride = 1, pad = 0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T #편향 넣기
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h,out_w, -1).tranpose(0,3,1,2)
        
        return out

In [10]:
x1 = np.random.rand(1,2,3,3)
x1_reshape = x1.reshape(1,-1)
print(x1)

[[[[0.81030361 0.65682833 0.65862817]
   [0.07602178 0.2137746  0.63161501]
   [0.32639641 0.31393715 0.7679785 ]]

  [[0.62617651 0.41044674 0.21868312]
   [0.57685879 0.06575494 0.7402571 ]
   [0.56907246 0.70190746 0.27674086]]]]


In [11]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        out = np.max(col, axis=1)
        
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)
        
        return out

In [12]:
# input_dim: 입력 데이터(채널 수, 높이, 너비)의 차원
# conv_param: 합성곱 계층의 초매개변수(dictionary)
# hidden_size: 은닉층의 뉴런 수 
# output_size: 출력층의 뉴런 수
# weight_init_std: 초기화 때의 가중치 표준편자 

class SimpleConvNet:
    def __init__(self, inout_dim=(1,28,28),
                conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride +1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        